# Collecting from the Spotify API #
Starting off, I called in the proper tools to help with getting in to the Spotify API.

In [31]:
import requests
import pandas as pd
import base64
import json
import urllib

In [32]:
Client_ID = pd.read_csv('SpotifyKeys20230919.txt')['Client_ID'].iloc[0]

In [33]:
Client_Secret = pd.read_csv('SpotifyKeys20230919.txt')['Client_Secret'].iloc[0]

In [34]:
client_cred = base64.b64encode(str(Client_ID + ":" + Client_Secret).encode("ascii"))

In [35]:
headers = {"Authorization": "Basic {}".format(client_cred.decode("ascii"))}

In [36]:
payload = {'grant_type' : 'client_credentials'}
url = 'https://accounts.spotify.com/api/token'

In [37]:
session_key_response = requests.post(url = url, data = payload, headers = headers)

In [38]:
session_key_response.status_code

200

In [39]:
session_header_key = session_key_response.json()

In [40]:
key = session_header_key['access_token']

In [41]:
session_headers = {"Authorization": "Bearer {}".format(key)}

### Calling the correct IDs ###
Now that I have gotten into the API, I can now use the proper IDs to gather the albums I want to use.

In [42]:
PFTW_id = '6ApYSpXF8GxZAgBTHDzYge'

In [43]:
PFTW_album_url = 'https://api.spotify.com/v1/albums/{}/tracks'.format(PFTW_id)

In [44]:
response_PFTW = requests.get(PFTW_album_url, headers = session_headers)

In [45]:
data_PFTW = json.loads(response_PFTW.text)

In [46]:
data_PFTW.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [47]:
data_PFTW['items'][0].keys()

dict_keys(['artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_urls', 'href', 'id', 'is_local', 'name', 'preview_url', 'track_number', 'type', 'uri'])

In [48]:
PFTW_track_ids = [x['id'] for x in data_PFTW['items']]

In [49]:
PFTW_track_ids

['3BDxv4ORzOpXAsa4lWVl4i',
 '76vMKwFtdDDCLcM6zXybjB',
 '22oEJW6r2rMb9z4IntfyEa',
 '1rqqCSm0Qe4I9rUvWncaom',
 '0wdV94AXGRphrbkDWLNzVC',
 '6PFoEnjp0PAe5siGswjH9i',
 '5trCWbYtza0QUxdmF9HAPX',
 '4hUR3thIs01mQUtVmZFJ8F',
 '50Hv5NZIM0puIUoBttjpfb',
 '18ULKfQl3KpCnklAKFfO2t',
 '18xmRkPWE0sGUzFyNNAAhN']

In [50]:
PFTW_track_strings = ','.join(str(x) for x in PFTW_track_ids)
PFTW_track_strings

'3BDxv4ORzOpXAsa4lWVl4i,76vMKwFtdDDCLcM6zXybjB,22oEJW6r2rMb9z4IntfyEa,1rqqCSm0Qe4I9rUvWncaom,0wdV94AXGRphrbkDWLNzVC,6PFoEnjp0PAe5siGswjH9i,5trCWbYtza0QUxdmF9HAPX,4hUR3thIs01mQUtVmZFJ8F,50Hv5NZIM0puIUoBttjpfb,18ULKfQl3KpCnklAKFfO2t,18xmRkPWE0sGUzFyNNAAhN'

Now with a string of all the song IDs, I can match it to the features for each and create a dataframe for them.

In [51]:
track_features_endpoint = "https://api.spotify.com/v1/audio-features?ids={}".format(PFTW_track_strings)


In [52]:
PFTW_features = requests.get(track_features_endpoint, headers = session_headers)


In [53]:
PFTW_features.status_code

200

In [54]:
PFTW_dict = json.loads(PFTW_features.text)

In [55]:
PFTW_dict['audio_features'][0]

{'danceability': 0.577,
 'energy': 0.939,
 'key': 1,
 'loudness': -1.352,
 'mode': 0,
 'speechiness': 0.108,
 'acousticness': 0.0272,
 'instrumentalness': 0,
 'liveness': 0.24,
 'valence': 0.714,
 'tempo': 105.092,
 'type': 'audio_features',
 'id': '3BDxv4ORzOpXAsa4lWVl4i',
 'uri': 'spotify:track:3BDxv4ORzOpXAsa4lWVl4i',
 'track_href': 'https://api.spotify.com/v1/tracks/3BDxv4ORzOpXAsa4lWVl4i',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3BDxv4ORzOpXAsa4lWVl4i',
 'duration_ms': 168613,
 'time_signature': 4}

In [56]:
PFTW_df = pd.DataFrame(PFTW_dict['audio_features'])
PFTW_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.577,0.939,1,-1.352,0,0.1080,0.0272,0.000000,0.2400,0.714,105.092,audio_features,3BDxv4ORzOpXAsa4lWVl4i,spotify:track:3BDxv4ORzOpXAsa4lWVl4i,https://api.spotify.com/v1/tracks/3BDxv4ORzOpX...,https://api.spotify.com/v1/audio-analysis/3BDx...,168613,4
1,0.491,0.867,6,-2.313,0,0.1030,0.1670,0.000000,0.4580,0.602,167.965,audio_features,76vMKwFtdDDCLcM6zXybjB,spotify:track:76vMKwFtdDDCLcM6zXybjB,https://api.spotify.com/v1/tracks/76vMKwFtdDDC...,https://api.spotify.com/v1/audio-analysis/76vM...,189187,4
2,0.577,0.833,5,-3.337,1,0.0695,0.0137,0.000000,0.1210,0.580,107.936,audio_features,22oEJW6r2rMb9z4IntfyEa,spotify:track:22oEJW6r2rMb9z4IntfyEa,https://api.spotify.com/v1/tracks/22oEJW6r2rMb...,https://api.spotify.com/v1/audio-analysis/22oE...,169667,4
3,0.579,0.904,5,-2.729,1,0.0618,0.1930,0.000000,0.0640,0.681,82.014,audio_features,1rqqCSm0Qe4I9rUvWncaom,spotify:track:1rqqCSm0Qe4I9rUvWncaom,https://api.spotify.com/v1/tracks/1rqqCSm0Qe4I...,https://api.spotify.com/v1/audio-analysis/1rqq...,190947,4
4,0.271,0.948,1,-2.207,1,0.2630,0.0467,0.000000,0.1630,0.515,83.190,audio_features,0wdV94AXGRphrbkDWLNzVC,spotify:track:0wdV94AXGRphrbkDWLNzVC,https://api.spotify.com/v1/tracks/0wdV94AXGRph...,https://api.spotify.com/v1/audio-analysis/0wdV...,186600,4
5,0.526,0.891,6,-1.549,1,0.0482,0.0375,0.000000,0.3750,0.804,94.095,audio_features,6PFoEnjp0PAe5siGswjH9i,spotify:track:6PFoEnjp0PAe5siGswjH9i,https://api.spotify.com/v1/tracks/6PFoEnjp0PAe...,https://api.spotify.com/v1/audio-analysis/6PFo...,219467,4
6,0.424,0.892,8,-3.780,0,0.1930,0.0426,0.000000,0.7110,0.762,174.008,audio_features,5trCWbYtza0QUxdmF9HAPX,spotify:track:5trCWbYtza0QUxdmF9HAPX,https://api.spotify.com/v1/tracks/5trCWbYtza0Q...,https://api.spotify.com/v1/audio-analysis/5trC...,198947,4
7,0.517,0.955,10,-1.848,0,0.1670,0.0224,0.000000,0.3640,0.610,132.951,audio_features,4hUR3thIs01mQUtVmZFJ8F,spotify:track:4hUR3thIs01mQUtVmZFJ8F,https://api.spotify.com/v1/tracks/4hUR3thIs01m...,https://api.spotify.com/v1/audio-analysis/4hUR...,177800,4
8,0.285,0.922,2,-1.932,0,0.2470,0.2370,0.000000,0.1590,0.472,198.063,audio_features,50Hv5NZIM0puIUoBttjpfb,spotify:track:50Hv5NZIM0puIUoBttjpfb,https://api.spotify.com/v1/tracks/50Hv5NZIM0pu...,https://api.spotify.com/v1/audio-analysis/50Hv...,160747,3
9,0.520,0.878,6,-1.768,0,0.0616,0.0133,0.000000,0.0433,0.783,168.033,audio_features,18ULKfQl3KpCnklAKFfO2t,spotify:track:18ULKfQl3KpCnklAKFfO2t,https://api.spotify.com/v1/tracks/18ULKfQl3KpC...,https://api.spotify.com/v1/audio-analysis/18UL...,166733,4


I will do the same with the Neotheater album

In [57]:
Neo_id = '02tIakRsIFGW8sO4pBtJgj'
album_tracks_endpoint = "https://api.spotify.com/v1/albums/{}/tracks".format(Neo_id)
Neo_tracks = requests.get(album_tracks_endpoint, headers = session_headers)
print(Neo_tracks.status_code)
Neo_dict = json.loads(Neo_tracks.text)
Neo_track_ids = [x['id'] for x in Neo_dict['items']]
Neo_track_strings = ','.join(str(x) for x in Neo_track_ids)
print(Neo_track_strings)
track_features_endpoint = "https://api.spotify.com/v1/audio-features?ids={}".format(Neo_track_strings)

200
5tVmNtMvCCABF6u2IyLLjX,7hBRNyuEkGR4qj5nTDGTry,17FK4etx1u2NK60cptINMS,61vlvG3pr4djrXPtHPdXII,4h1VLpsN8tl5J1c8nsrUCa,5MKfN1QC8ZWKeQGDFJSxaq,3HYgLawcb8IUGHHlHUQffN,3VygfAvvgVaJUeaBUSzlZu,4Z84bBOAu0dLVd2WimB3re,04gQHzS8FLPoNdnRKw4jDJ,4Dd0v2LBhMpqOwhmeU4yBJ,392snxgncTS2SKFgz64Lj3


In [58]:
Neo_features = requests.get(track_features_endpoint, headers = session_headers)
Neo_features.status_code

200

In [59]:
Neo_dict = json.loads(Neo_features.text)
Neo_dict['audio_features'][0]

{'danceability': 0.373,
 'energy': 0.569,
 'key': 10,
 'loudness': -7.846,
 'mode': 1,
 'speechiness': 0.0515,
 'acousticness': 0.205,
 'instrumentalness': 4.19e-05,
 'liveness': 0.377,
 'valence': 0.274,
 'tempo': 89.918,
 'type': 'audio_features',
 'id': '5tVmNtMvCCABF6u2IyLLjX',
 'uri': 'spotify:track:5tVmNtMvCCABF6u2IyLLjX',
 'track_href': 'https://api.spotify.com/v1/tracks/5tVmNtMvCCABF6u2IyLLjX',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5tVmNtMvCCABF6u2IyLLjX',
 'duration_ms': 256547,
 'time_signature': 4}

In [60]:
Neo_df = pd.DataFrame(Neo_dict['audio_features'])

In [61]:
Neo_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.373,0.569,10,-7.846,1,0.0515,0.20500,0.000042,0.377,0.274,89.918,audio_features,5tVmNtMvCCABF6u2IyLLjX,spotify:track:5tVmNtMvCCABF6u2IyLLjX,https://api.spotify.com/v1/tracks/5tVmNtMvCCAB...,https://api.spotify.com/v1/audio-analysis/5tVm...,256547,4
1,0.707,0.462,1,-6.696,1,0.1170,0.07320,0.000000,0.199,0.547,83.994,audio_features,7hBRNyuEkGR4qj5nTDGTry,spotify:track:7hBRNyuEkGR4qj5nTDGTry,https://api.spotify.com/v1/tracks/7hBRNyuEkGR4...,https://api.spotify.com/v1/audio-analysis/7hBR...,223948,4
2,0.626,0.639,10,-5.620,1,0.0850,0.04650,0.000000,0.161,0.638,144.009,audio_features,17FK4etx1u2NK60cptINMS,spotify:track:17FK4etx1u2NK60cptINMS,https://api.spotify.com/v1/tracks/17FK4etx1u2N...,https://api.spotify.com/v1/audio-analysis/17FK...,212783,4
3,0.629,0.583,2,-6.274,1,0.0415,0.14200,0.000000,0.118,0.492,147.036,audio_features,61vlvG3pr4djrXPtHPdXII,spotify:track:61vlvG3pr4djrXPtHPdXII,https://api.spotify.com/v1/tracks/61vlvG3pr4dj...,https://api.spotify.com/v1/audio-analysis/61vl...,250839,4
4,0.451,0.661,10,-6.961,0,0.0976,0.00321,0.000000,0.108,0.444,180.133,audio_features,4h1VLpsN8tl5J1c8nsrUCa,spotify:track:4h1VLpsN8tl5J1c8nsrUCa,https://api.spotify.com/v1/tracks/4h1VLpsN8tl5...,https://api.spotify.com/v1/audio-analysis/4h1V...,226336,4
5,0.431,0.190,9,-9.362,1,0.0371,0.85900,0.000000,0.100,0.128,76.925,audio_features,5MKfN1QC8ZWKeQGDFJSxaq,spotify:track:5MKfN1QC8ZWKeQGDFJSxaq,https://api.spotify.com/v1/tracks/5MKfN1QC8ZWK...,https://api.spotify.com/v1/audio-analysis/5MKf...,223201,4
6,0.646,0.626,5,-6.351,1,0.0729,0.01670,0.000009,0.112,0.372,83.981,audio_features,3HYgLawcb8IUGHHlHUQffN,spotify:track:3HYgLawcb8IUGHHlHUQffN,https://api.spotify.com/v1/tracks/3HYgLawcb8IU...,https://api.spotify.com/v1/audio-analysis/3HYg...,187263,4
7,0.528,0.727,6,-5.739,1,0.1790,0.05570,0.000000,0.176,0.212,155.927,audio_features,3VygfAvvgVaJUeaBUSzlZu,spotify:track:3VygfAvvgVaJUeaBUSzlZu,https://api.spotify.com/v1/tracks/3VygfAvvgVaJ...,https://api.spotify.com/v1/audio-analysis/3Vyg...,245270,4
8,0.590,0.682,0,-7.223,1,0.0564,0.01980,0.000000,0.178,0.665,125.055,audio_features,4Z84bBOAu0dLVd2WimB3re,spotify:track:4Z84bBOAu0dLVd2WimB3re,https://api.spotify.com/v1/tracks/4Z84bBOAu0dL...,https://api.spotify.com/v1/audio-analysis/4Z84...,198572,4
9,0.284,0.609,3,-6.033,1,0.1230,0.04380,0.000000,0.177,0.521,85.386,audio_features,04gQHzS8FLPoNdnRKw4jDJ,spotify:track:04gQHzS8FLPoNdnRKw4jDJ,https://api.spotify.com/v1/tracks/04gQHzS8FLPo...,https://api.spotify.com/v1/audio-analysis/04gQ...,196763,4


In [62]:
PFTW_df['album'] = ['PFTW'] * len(PFTW_df.index)

In [63]:
Neo_df['album'] = ['Neotheater'] * len(Neo_df.index)

In [64]:
audio_features = pd.concat([PFTW_df, Neo_df])

In [65]:
audio_features.to_csv('audio_features_Analysis.csv')